In [38]:
import urllib
import json
import sys
import pandas as pd
import time
import requests


In [ ]:
filename = sys.argv[1]
location = urllib.parse.quote(input("Put the place that you want to collect: "))
restaurants = get_store(location)
for restaurant in restaurants:
    get_review_set(restaurant)


In [7]:
def get_store(location):
    '''
    input: host_url ("https://www.mangoplate.com"), location ("대전 유성구 어은동", this should be converted into unicode)
    output: list containing review site (['https://www.mangoplate.com/restaurants/JVom8cKaKsBo', ... ])
    '''
    host_url = "https://www.mangoplate.com"
    # location = "%EB%8C%80%EC%A0%84%20%EC%9C%A0%EC%84%B1%EA%B5%AC%20%EC%96%B4%EC%9D%80%EB%8F%99"
    restaurant = []
    pageNum = 0
    while True:
        pageNum += 1
        url = host_url + "/search/" +  location + "?" + "keyword=" + location + "&" + "page=" + str(pageNum)
        r = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(r, "lxml")
        is_empty = True
        for link in soup.find_all('a'):
            href = link.get('href')
            if (href != None) and (href.startswith('/restaurants/')):
                temp = host_url + href
                is_empty = False
                if temp not in restaurant:
                    restaurant.append(temp)
        if (is_empty):
            break

    print("The number of stores:", len(restaurant))
    return restaurant

In [54]:
def get_review_set(restaurants):
    member_uuid = []
    member_nick_name = []
    review_rating = []
    reg_time = []
    comment = []
    restaurant_uuid = []
    restaurant_name = []
    restaurant_address = []
    restaurant_lat = []
    restaurant_long = []
    
    for restaurant in restaurants:
    
#         review_json_address = restaurants + "/reviews.json?request_count=9999&start_index=0"

        # Test Address
        review_json_address = 'https://stage.mangoplate.com/api/v5/restaurants/184801/reviews.json?access_token=&language=kor&request_count=99&start_index=0'
        
        review_json_data = json.loads(requests.get(review_json_address).text)
        for r in review_json_data:
                member_uuid.append(r["user"]["member_uuid"])
                member_nick_name.append(r["user"]["nick_name"])
                review_rating.append(r["action_value"])
                reg_time.append(r["time"])
                comment.append(r["comment"]["comment"])
                restaurant_uuid.append(r["restaurant"]["restaurant_uuid"])
                restaurant_name.append(r["restaurant"]["name"])
                restaurant_address.append(r["restaurant"]["address"])
                restaurant_lat.append(r["restaurant"]["latitude"])
                restaurant_long.append(r["restaurant"]["longitude"])
                
    return pd.DataFrame({'Restaurant ID': restaurant_uuid,
                         'Restaurant Name': restaurant_name,
                         'Restaurant Address': restaurant_address,
                         'Restaurant Latitude': restaurant_lat,
                         'Restaurant Longitude': restaurant_long,
                        'Member ID': member_uuid, 
                        'Member Nickname': member_nick_name,
                        'Rating': review_rating,
                        'Time': reg_time,
                        'Comment': comment})


In [60]:
test = get_review_set("JUST TEST")
test
# test.to_csv('test.csv', sep='\t', index=False, encoding='utf-8')

,Comment,Member ID,Member Nickname,Rating,Restaurant Address,Restaurant ID,Restaurant Latitude,Restaurant Longitude,Restaurant Name,Time
0,상수역 근처에 위치한 태국음식점. 약간의 웨이팅 후 매콤한 태국식 스프인 똠양꿍과 ...,35947,구현진,2,서울시 마포구 상수동 93-113,184801,37.54862033,126.92243353,뭄알로이,2017-03-12 11:59:54
1,상수/홍대 쪽에서 나름 유명한 타이음식점인 뭄알로이~\n2층에 위치하고 있으며 매장...,112555,키다리아저씨,2,서울시 마포구 상수동 93-113,184801,37.54862033,126.92243353,뭄알로이,2017-05-24 23:34:57
2,1.팟카오무쌉 2.팟타이 3.뿌팟퐁커리/ 망고에서 평이 좋은 뭄알로이. 역시 맛있다...,104601,퐝뎅,3,서울시 마포구 상수동 93-113,184801,37.54862033,126.92243353,뭄알로이,2015-12-07 18:01:17
3,가격대가 좀 있긴 하지만 맛있다.\n워낙 갈 곳이 많아 한 번 간 곳은\n두 번 가...,803877,Kate,3,서울시 마포구 상수동 93-113,184801,37.54862033,126.92243353,뭄알로이,2017-03-20 14:31:04
4,허름한 내부가 현지의 느낌을 살리려고 한건지 확실히 요즘 음식점의 느낌은 아님. 청...,26737,subing,2,서울시 마포구 상수동 93-113,184801,37.54862033,126.92243353,뭄알로이,2017-02-22 23:37:18
5,맛있어요! \n재밌는 식당이에요. 들어 가자마자 딱 태국의 식당 느낌이 들어요. 살...,259634,S E R A,3,서울시 마포구 상수동 93-113,184801,37.54862033,126.92243353,뭄알로이,2016-09-26 17:18:48
6,태국 현지 스멜 굿~ 뭄알로이\n\n태국 스멜이 입구부터 확 나는 뭄알로이 ! 새우...,112634,아지,3,서울시 마포구 상수동 93-113,184801,37.54862033,126.92243353,뭄알로이,2015-08-17 18:31:12
7,무난무난~ 맛있는 맛의 태국 음식점!\n가게 들어가면 되게 태국 느낌나요. 노래도...,453255,뿔뿔,3,서울시 마포구 상수동 93-113,184801,37.54862033,126.92243353,뭄알로이,2017-01-08 23:48:34
8,"농담이 아니고, 진심으로 한국에서 제일 맛있게 먹은 태국음식집! 친구랑 오후 2시쯤...",5821,R-Yoontz,3,서울시 마포구 상수동 93-113,184801,37.54862033,126.92243353,뭄알로이,2016-03-20 22:59:39
9,무난히 맛있다. 존맛은 아니고.홍대서 태국음식 먹고싶음 적당한가격대에 방문하기 좋은...,28187,jinee,2,서울시 마포구 상수동 93-113,184801,37.54862033,126.92243353,뭄알로이,2017-05-15 13:23:21
